# Archive Satellite Data Analysis (1984-2022)
TODO: add text

## 1. Set-up
### 1.1 Import modules

In [116]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import ee
import geemap

### 1.2 Authentification to Google Earth Engine

In [117]:
try:
    ee.Initialize()

except:
    ee.Authenticate()
    ee.Initialize()


Successfully saved authorization token.


## 2. Region of interest

In [118]:
latitude, longitude = 16.482743, 120.677159
point = ee.Geometry.Point([longitude, latitude])
dates = ['1984-01-01', '2022-01-10']

size = 0.022
polygon = ee.Geometry.Polygon([[[longitude-2*size, latitude+size],
            [longitude-2*size, latitude-size],
            [longitude+size, latitude-size],
            [longitude+size, latitude+size],
            [longitude-2*size, latitude+size]]], None, False)

In [119]:
S2 = 'COPERNICUS/S2_SR_HARMONIZED'
L9 = 'LANDSAT/LC09/C02/T1_L2'
L8 = 'LANDSAT/LC08/C02/T1_L2'

sat = S2

if sat == L8:
    cloud_label = 'CLOUD_COVER'

else:
    cloud_label = 'CLOUDY_PIXEL_PERCENTAGE'

imageCollection = ee.ImageCollection(sat).filterBounds(point).filterDate(dates[0], dates[1])
image = imageCollection.sort(cloud_label).first().clip(polygon)

In [120]:
Map = geemap.Map(center=[latitude, longitude], zoom=11)
Map.addLayer(image, vis_params={'bands': ['B4', 'B3', 'B2'], 'max':5000})
Map

Map(center=[16.482743, 120.677159], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

# DEM from GEE

In [121]:
dataset = ee.Image('USGS/SRTMGL1_003')#.clip(polygon)
elevation = dataset.select('elevation')
slope = ee.Terrain.slope(elevation)

# Get min and max values
minMaxVal = elevation.reduceRegion(reducer=ee.Reducer.minMax(), geometry=polygon)
print(minMaxVal.getInfo())

Map_DEM = geemap.Map(center=[latitude, longitude], zoom=11)
Map_DEM.addLayer(dataset.clip(polygon), {'min': 921, 'max': 1820, 'palette':['006633', 'EFFFCC', '662A00', 'D8D8D8', 'F5F5F5']}, 'DEM')


task_config = {
    'scale': 30,  
    'region': polygon
    }

task = ee.batch.Export.image(dataset, 'DEM_Philippines', task_config)

task.start()

#ee.batch.Export.image.toDrive({
  #'image': dataset,
  #'description': 'DEM_Sto_Nino',
  #'region': polygon
#})

Map_DEM

{'elevation_max': 1820, 'elevation_min': 971}


Map(center=[16.482743, 120.677159], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…